<a href="https://colab.research.google.com/github/BazanNicolas/Chabot-Fine-Tuning/blob/main/chabot_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [18]:
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Procesamiento del chat de WhatsApp para la creación del dataset

Primero debemos cargar un archivo de chat de WhatsApp (llamado chat.txt por defecto). Luego filtramos mensajes irrelevantes, como por ejemplo aquellos donde se encontraban mensajes multimedia, y separamos las conversaciones en prompts y responses. Además capturamos los mensajes del autor objetivo ("Nico Bazan" en este caso) y los emparejamos con los mensajes de los otros participantes del chat sin importar sus nombres para usarlos como entrenamiento.
Es importante notar que también se verifica que los mensajes no estén vacíos.

In [ ]:
# Datos irrelevantes a ignorar
irrelevantData = {
    'Eliminaste este mensaje.',
    'Se eliminó este mensaje.',
    '<Multimedia omitido>',
    'You deleted this message',
    'This message was deleted',
    '<Media omitted>'
}

# Verificar si el mensaje es irrelevante
def containsIrrelevantData(message):
    return any(irrelevant in message for irrelevant in irrelevantData)

# Procesar el chat con múltiples autores
def process_whatsapp_chat(filepath, target_author="Nico Bazan"):
    with open(filepath, 'r', encoding='utf-8') as file:
        chat_lines = file.readlines()

    prompts = []
    responses = []

    current_prompt = ""
    current_response = ""
    in_prompt = True  # Indica si estamos acumulando en el prompt

    for line in chat_lines:
        # Extraer fecha, autor y contenido del mensaje
        match = re.match(r'\d+/\d+/\d+,\s\d+:\d+\s-\s([^:]+):\s(.+)', line)

        if match:
            author = match.group(1).strip()
            message = match.group(2).strip()

            # Ignorar mensajes irrelevantes
            if containsIrrelevantData(message):
                continue

            if author != target_author:  # Otros autores
                if in_prompt:
                    current_prompt += f" {message}" if current_prompt else message
                else:
                    # Guardar el par si no está vacío
                    if current_prompt.strip() and current_response.strip():
                        prompts.append(current_prompt.strip())
                        responses.append(current_response.strip())
                    # Reiniciar para el siguiente ciclo
                    current_prompt = message
                    current_response = ""
                    in_prompt = True  # Volver a modo prompt
            else:  # Mensajes del autor objetivo (Nico Bazan)
                if not in_prompt:
                    current_response += f" {message}" if current_response else message
                else:
                    in_prompt = False  # Cambiar a modo respuesta
                    current_response = message

    # Guardar el último par si no está vacío
    if current_prompt.strip() and current_response.strip():
        prompts.append(current_prompt.strip())
        responses.append(current_response.strip())

    return prompts, responses

# Procesar el chat
chat_file = 'chat.txt'
target_author = 'Nico Bazan'
prompts, responses = process_whatsapp_chat(chat_file, target_author)

# Eliminar pares vacíos
prompts, responses = zip(*[
    (p, r) for p, r in zip(prompts, responses) if p.strip() and r.strip()
])

# Verificar los resultados
print("Ejemplo de Prompts:", prompts[:2])
print("Ejemplo de Responses:", responses[:2])


In [ ]:
# Imprimir 3 ejemplos limpios de prompts y responses
for i in range(min(3, len(prompts))):
    print(f"Prompt {i+1}: {prompts[i]}")
    print(f"Response {i+1}: {responses[i]}")
    print("-" * 40)

## Creación del archivo de entrenamiento con delimitadores

Convertiremos los prompts y responses procesados en un formato estructurado utilizando delimitadores claros (PROMPT, RESPONSE y ###). Cada par de conversación se guarda en el archivo train_data.txt para su posterior uso en el entrenamiento del modelo.

In [ ]:
# Crear archivo con delimitadores claros para prompt y response
train_data = ""
for prompt, response in zip(prompts, responses):
    train_data += f"PROMPT:\n{prompt}\nRESPONSE:\n{response}\n###\n"  # Delimitadores claros

# Guardar los datos en 'train_data.txt'
with open('train_data.txt', 'w', encoding='utf-8') as f:
    f.write(train_data)

print("Datos guardados correctamente en 'train_data.txt'.")


## Carga y configuración del modelo GPT-2 en español

Ahora realizamos la configuración del modelo GPT-2 en español que será fine tuneado utilizando la librería transformers. Además, se ajustan parámetros como:

* Carga del modelo base y el tokenizador GPT-2 con un model_max_length de 64.
* Configuración del dropout (comentada inicialmente, pero se puede ajustar para el entrenamiento).
* Alineación del pad_token con `eos_token,** garantizando que el padding no interfiera con el procesamiento del modelo.
* Ajuste de embeddings: Se asegura de que la dimensión del vocabulario se adapte al tokenizador cargado.
* Sincronización del peso de la capa de salida (lm_head) con los embeddings iniciales para evitar inconsistencias.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo base en español
model_name = "DeepESP/gpt2-spanish"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=64)

# Configurar dropout (esto se aplicará durante el entrenamiento)
# model.config.attn_pdrop = 0.3
# model.config.embd_pdrop = 0.3
# model.config.resid_pdrop = 0.3


# Alinear el token de padding con el token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))  # Ajustar embeddings

# Inicializa el peso de la lm_head si es necesario
with torch.no_grad():
    model.lm_head.weight = model.transformer.wte.weight


## Carga y creación del dataset para entrenamiento y validación

Esta celda carga los datos desde el archivo train_data.txt, los procesa utilizando delimitadores claros y los convierte en un dataset compatible con Hugging Face. A continuación, el dataset se divide en subconjuntos para entrenamiento y validación (80-20), garantizando que los datos se mezclen aleatoriamente.

In [ ]:
import re
from datasets import Dataset, DatasetDict

# Función para leer el archivo y agrupar los ejemplos con delimitadores claros
def load_grouped_messages(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        raw_data = f.read()

    # Usar regex para extraer prompts y responses con delimitadores claros
    examples = re.findall(r"PROMPT:\n(.*?)\nRESPONSE:\n(.*?)\n###", raw_data, re.DOTALL)

    if not examples:
        raise ValueError("No se encontraron ejemplos en el archivo. Verifica el formato.")

    # Crear lista de diccionarios con prompts y responses separados
    data = [{"prompt": prompt.strip(), "response": response.strip()} for prompt, response in examples]
    return data

# Cargar los datos desde el archivo
data = load_grouped_messages("train_data.txt")

# Crear un Dataset de Huggingface a partir de los datos cargados
dataset = Dataset.from_dict({
    "prompt": [d["prompt"] for d in data],
    "response": [d["response"] for d in data]
})

# Dividir el dataset en entrenamiento y validación, asegurando mezcla
train_test_split = dataset.train_test_split(test_size=0.2, shuffle=True)

# Crear un DatasetDict para entrenamiento y validación
datasets = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"]
})

# Verificar la estructura del dataset
print("Primeros ejemplos de entrenamiento:\n", datasets["train"][:2])


## Definición de funciones auxiliares para tokenización y agrupación de textos

Esta celda define dos funciones auxiliares esenciales para la preparación del dataset y el entrenamiento del modelo. Estas funciones se encargan de tokenizar los textos y agruparlos en bloques del tamaño adecuado para optimizar el entrenamiento del modelo.
Devuelven un nuevo dataset en el que los textos están tokenizados (convertidos en índices del vocabulario) y se incluye una máscara de atención para distinguir los tokens relevantes de los tokens de padding. Además, se preparan labels para el entrenamiento.

(Créditos a [Cristian Cardellino](https://crscardellino.net) por estas [funciones](https://huggingface.co/crscardellino/flisol-cba-martin-fierro/resolve/main/utils.py))

In [24]:
from datasets import DatasetDict
from transformers import PreTrainedTokenizerBase
from typing import Callable, Dict, List


def tokenize(
    tokenizer: PreTrainedTokenizerBase, end_char: str = "\n"
) -> Callable[[Dict[str, List[str]]], DatasetDict]:
    """
    Tokeniza los textos agregando un caracter final opcional (`end_char`).
    """

    def _tokenize(examples: Dict[str, List[str]]) -> DatasetDict:
        # Asegurarse de que cada entrada tiene texto válido
        texts = [f"{e}{end_char}" for e in examples["text"] if e.strip()]
        return tokenizer(texts, truncation=True, padding=True)

    return _tokenize


def group_texts(examples: Dict[str, List[int]], block_size: int = 128) -> Dict[str, List[int]]:
    """
    Agrupa los textos en bloques del tamaño especificado y genera etiquetas (labels).
    """

    # Concatenar todos los tokens
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples["input_ids"])

    if total_length < block_size:
        raise ValueError("Los datos son insuficientes para formar un bloque del tamaño especificado.")

    # Ajustar el total_length para evitar remainders incompletos
    total_length = (total_length // block_size) * block_size

    # Dividir en bloques del tamaño especificado
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }

    # Copiar input_ids como labels para el entrenamiento
    result["labels"] = result["input_ids"].copy()

    return result


## Tokenización del dataset para preparación del entrenamiento

En esta celda se realiza la tokenización de los datos de entrenamiento y validación, preparando los textos para que puedan ser procesados por el modelo. La función de tokenización transforma los textos en índices del vocabulario y añade padding para mantener la consistencia en la longitud de los ejemplos.

In [ ]:
from functools import partial

'''
Propósito: Combina los prompts y responses en un solo string, los tokeniza y asegura que todos los ejemplos tengan la misma longitud (128 tokens).
Truncamiento: Corta los textos si superan el límite definido (128 tokens).
Padding: Añade tokens de relleno hasta alcanzar 128 tokens para garantizar que todos los ejemplos tengan la misma longitud.
Retorno: Devuelve los tensores en formato PyTorch (return_tensors="pt").
'''
def tokenize_function(examples):
    # Concatenar prompt y response en un solo string
    texts = [f"{p} {r}".strip() for p, r in zip(examples["prompt"], examples["response"])]
    return tokenizer(
        texts,
        truncation=True,  # Truncar si excede el límite
        max_length=128,    # Limitar a 128 tokens
        padding="max_length",  # Rellenar hasta 128 tokens
        return_tensors="pt"
    )

# Usar partial para fijar el tokenizador en la función
tokenize_with_tokenizer = partial(tokenize_function)

# Aplicar la tokenización al dataset usando el partial
tokenized_datasets = datasets.map(
    tokenize_with_tokenizer,  # Usar la función parcial
    batched=True,
    num_proc=4,
    remove_columns=["prompt", "response"]  # Remover columnas originales
)

# Verificar los primeros datos tokenizados
print("Datos tokenizados:\n", tokenized_datasets["train"][:2])


## Agrupación de textos en bloques del tamaño especificado

Se aplica la función group_texts al dataset previamente tokenizado para dividir los tokens en bloques del tamaño especificado (por defecto, 128 tokens). Esto asegura que los lotes (batches) tengan una longitud uniforme durante el entrenamiento.

In [ ]:
# Agrupar los textos en bloques del tamaño especificado
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=1  # Cambiar a 1 si estás usando Google Colab sin núcleos adicionales
)

# Verificar los primeros ejemplos agrupados (mostrar solo los primeros 20 tokens)
print("Datos agrupados (primeros 20 tokens):\n",
      {k: v[:20] for k, v in lm_datasets["train"][:2].items()})


## Configuración del entrenamiento y optimización del modelo

Se configura los parámetros clave para el entrenamiento del modelo mediante el uso de TrainingArguments y el objeto Trainer. Se define el optimizador AdamW con un scheduler cosine para ajustar dinámicamente la tasa de aprendizaje. Además, se calcula el número de pasos totales y de calentamiento (warmup) para controlar la tasa al inicio del entrenamiento. La configuración incluye la opción de EarlyStopping para detener el entrenamiento si no hay mejora en la pérdida, optimizando así los recursos.

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, get_scheduler

# Argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir="./fine_tuned_gpt2",
    evaluation_strategy="epoch",  # Evaluar después de cada época
    save_strategy="epoch",  # Guardar el modelo al final de cada época
    num_train_epochs=3,  # Número de épocas
    per_device_train_batch_size=8,  # Tamaño del batch
    learning_rate=1e-5,  # Tasa de aprendizaje
    weight_decay=0.01,  # Regularización
    logging_steps=5,  # Log cada 5 pasos
    report_to=[],  # Deshabilitar W&B
    load_best_model_at_end=True,  # Cargar el mejor modelo al final
    metric_for_best_model="loss",  # Usar 'loss' como métrica principal
    greater_is_better=False  # Minimizar la pérdida
)

# Definir el optimizador usando los argumentos
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

# Calcular el número total de pasos de entrenamiento
num_training_steps = (
    len(lm_datasets["train"]) // training_args.per_device_train_batch_size
) * training_args.num_train_epochs

num_warmup_steps = int(num_training_steps * 0.1)  # Warmup del 10%
scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=int(num_training_steps * 0.1),
    num_training_steps=num_training_steps
)


# Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    optimizers=(optimizer, scheduler),  # Optimizer y scheduler
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]  # Detener si no mejora
)


## Inicio del Entrenamiento del Modelo
Ejecuta el proceso de fine-tuning

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.161100,1.051037
2,1.189500,1.019949
3,0.912100,1.016659


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=6699, training_loss=1.0928075167292497, metrics={'train_runtime': 2334.4003, 'train_samples_per_second': 22.952, 'train_steps_per_second': 2.87, 'total_flos': 3500006768640000.0, 'train_loss': 1.0928075167292497, 'epoch': 3.0})

## Evaluación del Modelo: Perplejidad y Entropía Cruzada (No ejecutar por ahora, Cuda se queda sin memoria 😭)

Esta celda evalúa el rendimiento del modelo mediante dos métricas: perplejidad y entropía cruzada. La perplejidad se calcula a partir de la pérdida en el conjunto de validación y refleja qué tan bien el modelo predice secuencias de texto. La entropía cruzada mide la discrepancia entre las distribuciones de los logits del modelo(predicciones sin procesar) y las etiquetas reales.
En otras palabras, mide cuán diferentes son estas dos distribuciones: la predicción del modelo (logits) y la distribución "objetivo" (la palabra correcta con probabilidad 1). Una

In [ ]:
import math
import torch.nn.functional as F
import torch
import gc

def compute_perplexity(eval_loss):
    """Calcula la perplejidad a partir de la pérdida."""
    return math.exp(eval_loss)

def compute_cross_entropy(logits, labels):
    """Calcula la entropía cruzada entre los logits y las etiquetas reales."""
    shift_logits = logits[..., :-1, :].contiguous()  # Desplazar logits
    shift_labels = labels[..., 1:].contiguous()  # Desplazar etiquetas

    loss = F.cross_entropy(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1)
    )
    return loss.item()

# Liberar memoria ANTES de la evaluación
torch.cuda.empty_cache()
gc.collect()

# Evaluar el modelo con batches pequeños para evitar problemas de memoria
valid_dataset = lm_datasets["validation"]
batch_size = 2  # Ajustar según la memoria disponible

logits_list = []
labels_list = []

with torch.no_grad():
    for i in range(0, len(valid_dataset), batch_size):
        batch = valid_dataset[i: i + batch_size]

        # Preparar inputs y moverlos a GPU
        inputs = {
            "input_ids": torch.tensor(batch["input_ids"]).to(device),
            "attention_mask": torch.tensor(batch["attention_mask"]).to(device),
        }
        labels = torch.tensor(batch["labels"]).to(device)

        # Generar logits y moverlos inmediatamente a la CPU para liberar memoria GPU
        outputs = model(**inputs)
        logits_list.append(outputs.logits.cpu())
        labels_list.append(labels.cpu())

        # Liberar memoria GPU después de cada lote
        torch.cuda.empty_cache()
        gc.collect()

# Concatenar todos los logits y etiquetas
logits = torch.cat(logits_list, dim=0)
all_labels = torch.cat(labels_list, dim=0)

# Calcular la entropía cruzada con los logits concatenados
cross_entropy = compute_cross_entropy(logits, all_labels)
print(f"Entropía Cruzada: {cross_entropy}")

# Evaluar el modelo usando el Trainer y mostrar la perplejidad
eval_result = trainer.evaluate()
eval_loss = eval_result["eval_loss"]

print(f"Pérdida de validación: {eval_loss}")
print(f"Perplejidad: {compute_perplexity(eval_loss)}")

# Liberar memoria al final
torch.cuda.empty_cache()
gc.collect()


## Guardar el modelo fine-tuneado

Una vez finalizado el entrenamiento, guarda el modelo fine-tuneado y su tokenizador para reutilizarlos más adelante.

In [ ]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

## Generación y Comparación de Respuestas entre el Modelo Base y el Modelo Fine-Tuneado

Esta celda implementa una función para generar respuestas del modelo y comparar los resultados entre el modelo base y el fine-tuneado. Se utiliza la misma entrada (prompt) en ambos modelos para observar cómo el fine tunning mejora (o cambia) las respuestas.

In [ ]:
# Función para generar una respuesta
def generate_response(prompt, model, tokenizer, max_length=50):
    model.eval()  # Asegurarse de que el modelo esté en modo evaluación
    inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

    # Generar salida del modelo
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        top_p=0.8,  # Controlar diversidad
        top_k=30,   # Opciones más probables
        temperature=0.7,  # Controlar aleatoriedad
        do_sample=True  # Habilitar muestreo aleatorio
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()

# Cargar ambos modelos: base y fine-tuneado
base_model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish").to(device)
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt2").to(device)

# Comparar respuestas entre ambos modelos
def compare_models(prompt):
    print(f"Prompt: {prompt}")

    # Generar respuesta con el modelo base
    base_response = generate_response(prompt, base_model, tokenizer)
    print(f"Respuesta del Modelo Base:\n{base_response}\n")

    # Generar respuesta con el modelo fine-tuneado
    fine_tuned_response = generate_response(prompt, fine_tuned_model, tokenizer)
    print(f"Respuesta del Modelo Fine-Tuneado:\n{fine_tuned_response}")

    print("-" * 40)

# Ejemplo de comparación con un prompt personalizado
prompt = "Hola Nico, cómo estás? Te queria pregunar algo"
compare_models(prompt)

# Comparar respuestas usando prompts limpios del dataset
for i in range(2):
    prompt = prompts[i]
    compare_models(prompt)


## Chat Interactivo con el Modelo Fine-Tuneado

 Acá se implementa un chat interactivo con el modelo fine-tuneado, permitiendonos conversar directamente con el chat bot. El ciclo permanece activo hasta que escribamos 'terminar', momento en el que la conversación finaliza.

In [ ]:
print("Bienvenido al chat con el bot. Escribe 'terminar' para finalizar la conversación.")

while True:
    user_input = input("Vos: ")  # Capturar entrada del usuario
    if user_input.lower() == "terminar":
        print("Bot: Hasta luego!")
        break  # Finaliza el chat si el usuario escribe "terminar"

    # Generar la respuesta del bot con longitud máxima de 50 tokens
    response = generate_response(user_input, model, tokenizer, max_length=50)
    print(f"Bot: {response}")
